In [ ]:
!git clone -b unet-gan https://github.com/hoangbros03/GAN-implement.git
!pip install torch-fidelity

In [ ]:
import os
os.chdir('GAN-implement')
os.chdir('evaluate')
from evaluate import frechet_inception_distance, convert_dataloader_to_sample, convert_rgb
os.chdir('../src/pure_gan')
from mnist_dataloader import check_and_process_dataloader
os.chdir('../cnn_gan')
from train import Trainer
dataloader = check_and_process_dataloader("mnist", (1, 28, 28), 32)

In [ ]:
device = 'cuda'
trainer = Trainer(device=device, generator="mnist", discriminator="mnist", img_size=28, channel=1)
trainer.train_loop(dataloader, num_epochs=1, D_G_train_proportion=3)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torchvision.utils as vutils

# Plot the fake images from the last epoch
plt.figure(figsize=(15, 15))
plt.axis("off")
plt.title("Generated Images")
plt.imshow(np.transpose(trainer.img_list[1],(1,2,0)))
plt.savefig('mnist')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(trainer.G_losses,label="G")
plt.plot(trainer.D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()